In [1]:
import pandas as pd
import re


1. Load data
2. Clean data (Million song subset)
3. Merge datasets
4. Save data

In [2]:
million_songs_df = pd.read_csv('../data/raw/2_million_song_subset_raw.csv')
hot100_df = pd.read_csv('../data/raw/1_hot100.csv')

million_songs_df.head()

,title;artist
0,b'Je Sais Que La Terre Est Plate';b'Rapha\xc3\...
1,b'On Efface';b'Julie Zenatti'
2,b'Howells Delight';b'The Baltimore Consort'
3,b'Martha Served';b'I Hate Sally'
4,b'Zip-A-Dee-Doo-Dah (Song of the South)';b'Orl...


In [3]:
missing_columns = million_songs_df.isna().any()
print(missing_columns)


title;artist    False
dtype: bool


Split million_songs_df into songs and artists

In [4]:
million_songs_df[['song_title', 'artist']] = million_songs_df['title;artist'].str.split(';b', expand=True)  # separate title and artist into separate columns

# clean strings
million_songs_df['song_title'] = million_songs_df['song_title'].str.replace(r"^b['\"]|['\"]$", "", regex=True)
million_songs_df['artist'] = million_songs_df['artist'].str.replace(r"^['\"]|['\"]$", "", regex=True)

# decode UTF-8 characters
def decode_utf8(x):
    if pd.isna(x):
        return x
    if '\\x' in x:
        return bytes(x, 'utf-8').decode('unicode_escape').encode('latin1').decode('utf-8')
    return x

million_songs_df['song_title'] = million_songs_df['song_title'].apply(decode_utf8)
million_songs_df['artist'] = million_songs_df['artist'].apply(decode_utf8)

million_songs_df = million_songs_df.drop('title;artist', axis=1)

# million_songs_df.head()

In [5]:
million_songs_df.head(30)


,song_title,artist
0,Je Sais Que La Terre Est Plate,Raphaël
1,On Efface,Julie Zenatti
2,Howells Delight,The Baltimore Consort
3,Martha Served,I Hate Sally
4,Zip-A-Dee-Doo-Dah (Song of the South),Orlando Pops Orchestra
5,Liquid Time (composition by John Goodsall),Brand X
6,Misery Path (From the Privilege of Evil),Amorphis
7,Nuovi Re pt. II (feat. Tek money - Lady Tambler),Inoki
8,Halloween,Dead Kennedys
9,Parto em terras distantes,Brigada Victor Jara


Remove second parentheses in song_title


In [6]:
def remove_parentheses(text):
    return re.sub(r'\(.*?\)', '', text)  # remove everything inside parentheses

million_songs_df['song_title'] = million_songs_df['song_title'].apply(remove_parentheses)

million_songs_df.head(30)

,song_title,artist
0,Je Sais Que La Terre Est Plate,Raphaël
1,On Efface,Julie Zenatti
2,Howells Delight,The Baltimore Consort
3,Martha Served,I Hate Sally
4,Zip-A-Dee-Doo-Dah,Orlando Pops Orchestra
5,Liquid Time,Brand X
6,Misery Path,Amorphis
7,Nuovi Re pt. II,Inoki
8,Halloween,Dead Kennedys
9,Parto em terras distantes,Brigada Victor Jara


Check for duplicates

In [7]:
million_songs_df.duplicated(subset=['song_title', 'artist']).sum()

million_songs_df.drop_duplicates(subset=['song_title', 'artist'], keep='first')

display(million_songs_df)


,song_title,artist
0,Je Sais Que La Terre Est Plate,Raphaël
1,On Efface,Julie Zenatti
2,Howells Delight,The Baltimore Consort
3,Martha Served,I Hate Sally
4,Zip-A-Dee-Doo-Dah,Orlando Pops Orchestra
...,...,...
9995,One About Heaven,Brent Lamb
9996,October,U2
9997,Comin' Home,ZO2
9998,Pode Me Chamar,Eddie


## Clean hot100_df

In [8]:
hot100_df['song_title'] = hot100_df['song_title'].apply(remove_parentheses)

In [9]:
hot100_df.head(30)

,position,song_title,artist
0,1,Die With A Smile,Lady Gaga & Bruno Mars
1,2,A Bar Song,Shaboozey
2,3,Birds Of A Feather,Billie Eilish
3,4,Lose Control,Teddy Swims
4,5,APT.,ROSE & Bruno Mars
5,6,That's So True,Gracie Abrams
6,7,Luther,Kendrick Lamar & SZA
7,8,I Had Some Help,Post Malone Featuring Morgan Wallen
8,9,Espresso,Sabrina Carpenter
9,10,Taste,Sabrina Carpenter


Clean string columns

In [10]:
def clean_string_columns(df):
        for column in df.columns:
            if df[column].dtype == 'object':
                df[column] = df[column].str.lower()  # Convert to lowercase
                df[column] = df[column].str.replace('feat.', 'featuring', regex=False)  # Replace 'feat.' with 'featuring'
                df[column] = df[column].str.replace('&', 'and', regex=False)  # Replace '&' with 'and'
        return df

In [11]:
hot100_df = clean_string_columns(hot100_df)
hot100_df.head(30)

,position,song_title,artist
0,1,die with a smile,lady gaga and bruno mars
1,2,a bar song,shaboozey
2,3,birds of a feather,billie eilish
3,4,lose control,teddy swims
4,5,apt.,rose and bruno mars
5,6,that's so true,gracie abrams
6,7,luther,kendrick lamar and sza
7,8,i had some help,post malone featuring morgan wallen
8,9,espresso,sabrina carpenter
9,10,taste,sabrina carpenter


In [12]:
million_songs_df = clean_string_columns(million_songs_df)
million_songs_df.head(30)

,song_title,artist
0,je sais que la terre est plate,raphaël
1,on efface,julie zenatti
2,howells delight,the baltimore consort
3,martha served,i hate sally
4,zip-a-dee-doo-dah,orlando pops orchestra
5,liquid time,brand x
6,misery path,amorphis
7,nuovi re pt. ii,inoki
8,halloween,dead kennedys
9,parto em terras distantes,brigada victor jara


check matching columns

In [16]:
# Function to clean text: remove punctuation and convert to lowercase
def clean_text_for_matching(text):
    # Remove all punctuation and convert to lowercase
    return re.sub(r'[^\w\s]', '', str(text).lower())

# Create clean versions of song titles for matching
hot100_clean = hot100_df['song_title'].apply(clean_text_for_matching)
million_clean = million_songs_df['song_title'].apply(clean_text_for_matching)

# Find matches using cleaned titles
matches = pd.merge(
    pd.DataFrame({
        'clean_title': hot100_clean,
        'original_title': hot100_df['song_title'],
        'artist': hot100_df['artist']
    }),
    pd.DataFrame({
        'clean_title': million_clean,
        'million_title': million_songs_df['song_title'],
        'million_artist': million_songs_df['artist']
    }),
    on='clean_title'
)

print(f"Number of matches after cleaning: {len(matches)}")
print("\nExample matches:")
display(matches[['original_title', 'million_title', 'artist', 'million_artist']].head(20))

print(f"\nTotal songs in hot100_df: {len(hot100_df)}")
print(f"Total songs in million_songs_df: {len(million_songs_df)}")

Number of matches after cleaning: 14

Example matches:


,original_title,million_title,artist,million_artist
0,smile,smile,morgan wallen,marvin sapp
1,smile,smile,morgan wallen,the london pops orchestra
2,smile,smile,morgan wallen,heavenly
3,smile,smile,morgan wallen,xcultures
4,wildflower,wildflower,billie eilish,bon jovi
5,good news,good news,shaboozey,kingston trio
6,drive,drive,sza,the cars
7,drive,drive,sza,pacha massive
8,help me,help me,real boston richey,chris brown
9,bad dreams,bad dreams,teddy swims,lyrics born



Total songs in hot100_df: 100
Total songs in million_songs_df: 10000
